In [1]:
from google.colab import drive
import sys
drive.mount('/mnt/drive')

storage_path = "/mnt/drive/MyDrive/cs224u_final_project"
sys.path.append(storage_path)
from compgen import recogs_exact_match

Mounted at /mnt/drive


In [2]:
%%capture
!pip install wget openai datasets pandas pyarrow
import pandas as pd
import numpy as np

In [3]:
testing_examples_df = pd.read_parquet(f"{storage_path}/test_data_200_v2.parquet")
testing_examples_df.head()

,input,output,category,output_renum,prompt_test_base,prompt_test_guide
9202,A lion squeezed a balloon on the stage .,lion ( 46 ) ; balloon ( 44 ) ; * stage ( 49 ) ...,passive_to_active,lion ( 1 ) ; balloon ( 2 ) ; * stage ( 3 ) ; n...,<s>[INST] A lion squeezed a balloon on the sta...,<s>[INST] You are given a sentence and must co...
20450,Emma squeezed the professor on the hanger .,Emma ( 48 ) ; * professor ( 59 ) ; * hanger ( ...,passive_to_active,Emma ( 1 ) ; * professor ( 2 ) ; * hanger ( 3 ...,<s>[INST] Emma squeezed the professor on the h...,<s>[INST] You are given a sentence and must co...
14358,The girl squeezed Liam .,* girl ( 43 ) ; Liam ( 56 ) ; squeeze ( 20 ) A...,passive_to_active,* girl ( 1 ) ; Liam ( 2 ) ; squeeze ( 3 ) AND ...,<s>[INST] The girl squeezed Liam . [/INST],<s>[INST] You are given a sentence and must co...
12832,The teacher in the cart cooked Emma .,* teacher ( 31 ) ; * cart ( 6 ) ; Emma ( 47 ) ...,obj_pp_to_subj_pp,* teacher ( 1 ) ; * cart ( 2 ) ; Emma ( 3 ) ; ...,<s>[INST] The teacher in the cart cooked Emma ...,<s>[INST] You are given a sentence and must co...
2052,Michael believed that Mason hoped that the con...,Michael ( 15 ) ; Mason ( 37 ) ; * consumer ( 4...,cp_recursion,Michael ( 6 ) ; Mason ( 2 ) ; * consumer ( 3 )...,<s>[INST] Michael believed that Mason hoped th...,<s>[INST] You are given a sentence and must co...


In [4]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-hocl9czb/unsloth_b422623220434daf9cc6f22a9b5651ea
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-hocl9czb/unsloth_b422623220434daf9cc6f22a9b5651ea
  Resolved https://github.com/unslothai/unsloth.git to commit d3a33a0dc3cabd3b3c0dba0255fb4919db44e3b5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.0 MB/s eta 0:00:00
  Created wheel for unsloth: filename=unsloth-2024.3-py3-none-any.whl size=94476 sha256=afda56f0864308061036dd4b2e2d6317c73c614881a4a9d2a281aa65061c4050
  Stored in directory: /tmp/pip-ephem-wheel-cache-9yjlff41/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222

In [5]:
import gc
from unsloth import FastLanguageModel
import torch
from tqdm import tqdm
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

inference_params = [
    ("guide", 0, 0),
    # ("guide", 161, 200),
    ("base", 0, 0),
    # ("base", 161, 200),
    # ("guide", 100, 62),
    # ("guide", 100, 120),
    # ("base", 100, 120),
    # ("base", 100, 62),
    # ("guide", 50, 120),
    # ("guide", 50, 62),
    # ("base", 50, 120),
    # ("base", 50, 62),
]

for inf_type, data_num, checkpoint in inference_params:
  print("Working on:", inf_type, data_num, checkpoint)
  testing_examples_df = pd.read_parquet(f"{storage_path}/test_data_200.parquet")
  try:
    del model, tokenizer
    gc.collect()
    torch.cuda.empty_cache()
  except:
    pass

  if data_num != 0:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = f"/mnt/drive/MyDrive/cs224u_final_project/model_{inf_type}_{data_num}/checkpoint-{checkpoint}/",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )
  else:
    model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit)

  tokenizer.padding_side = "left"
  tokenizer.pad_token = tokenizer.eos_token

  FastLanguageModel.for_inference(model)

  if inf_type == "base":
    num_chunks = 10
  else:
    num_chunks = 20
  df_chunks = np.array_split(testing_examples_df, num_chunks)

  for df in tqdm(df_chunks, total=len(df_chunks)):
      encoding = tokenizer(df[f"prompt_test_{inf_type}"].to_list(), padding=True, return_tensors='pt').to("cuda")
      generated_ids = model.generate(**encoding, max_new_tokens = 1024, pad_token_id=tokenizer.eos_token_id) #, use_cache = True
      testing_examples_df.loc[df.index, f"gen_test_{inf_type}"] = tokenizer.batch_decode(generated_ids)

  if inf_type == "base":
    testing_examples_df.loc[testing_examples_df.gen_test_base.str.len().eq(1), "gen_test_base"] = testing_examples_df.loc[testing_examples_df.gen_test_base.str.len().eq(1), "gen_test_base"].str[0]
    testing_examples_df["result_base"] = testing_examples_df.gen_test_base.str.split(pat="\[\/INST\]").str[-1].str.replace("</s>", "").str.strip()
    testing_examples_df["base_correct"] = testing_examples_df.apply(lambda row: recogs_exact_match(row["output_renum"], row["result_base"]), axis=1)
  else:
    testing_examples_df.loc[testing_examples_df.gen_test_guide.str.len().eq(1), "gen_test_guide"] = testing_examples_df.loc[testing_examples_df.gen_test_guide.str.len().eq(1), "gen_test_guide"].str[0]
    testing_examples_df["result_guide"] = testing_examples_df.gen_test_guide.str.split("Step 5: Represent the logical form").str[-1].str.replace("</s>", "").str.strip()
    testing_examples_df["guide_correct"] = testing_examples_df.apply(lambda row: recogs_exact_match(row["output_renum"], row["result_guide"]), axis=1)

  testing_examples_df.to_parquet(f"{storage_path}/model_{inf_type}_{data_num}_{checkpoint}_preds.parquet")
  print(testing_examples_df[f"{inf_type}_correct"].mean())

Working on: guide 0 0


config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

100%|██████████| 20/20 [51:43<00:00, 155.17s/it]


0.135
Working on: base 0 0
==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


100%|██████████| 10/10 [13:01<00:00, 78.20s/it]

0.0


# Inference base

In [6]:
# FastLanguageModel.for_inference(model)

# # encoding = tokenizer(testing_examples_df["prompt_test_base"].to_list(), padding=True, return_tensors='pt').to("cuda")
# # generated_ids = model.generate(**encoding, max_new_tokens = 1024, use_cache = True, pad_token_id=tokenizer.eos_token_id)
# # testing_examples_df["gen_test_base"] = tokenizer.batch_decode(generated_ids)
# count=0
# for id, row in testing_examples_df.iterrows():
#   if count % 10 == 0:
#     print(count)
#   encoding = tokenizer([row["prompt_test_base"]], return_tensors='pt').to("cuda")
#   generated_ids = model.generate(**encoding, max_new_tokens = 1024, use_cache = True, pad_token_id=tokenizer.eos_token_id)
#   testing_examples_df.loc[id, "gen_test_base"] = tokenizer.batch_decode(generated_ids)
#   count += 1


In [7]:
# testing_examples_df.loc[testing_examples_df.gen_test_base.str.len().eq(1), "gen_test_base"] = testing_examples_df.loc[testing_examples_df.gen_test_base.str.len().eq(1), "gen_test_base"].str[0]
# testing_examples_df["result_base"] = testing_examples_df.gen_test_base.str.split(pat="\[\/INST\]").str[-1].str.replace("</s>", "").str.strip()
# testing_examples_df["base_correct"] = testing_examples_df.apply(lambda row: recogs_exact_match(row["output_renum"], row["result_base"]), axis=1)
# testing_examples_df.to_parquet(f"{storage_path}/model_base_170_105_preds.parquet")
# testing_examples_df["base_correct"].mean()

# Inference guide

In [8]:
# FastLanguageModel.for_inference(model)

# count=0
# for id, row in testing_examples_df.iterrows():
#   if count % 10 == 0:
#     print(count)
#   encoding = tokenizer([row["prompt_test_guide"]], return_tensors='pt').to("cuda")
#   generated_ids = model.generate(**encoding, max_new_tokens = 1024, use_cache = True, pad_token_id=tokenizer.eos_token_id)
#   testing_examples_df.loc[id, "gen_test_guide"] = tokenizer.batch_decode(generated_ids)
#   count += 1


In [9]:
# testing_examples_df.loc[testing_examples_df.gen_test_guide.str.len().eq(1), "gen_test_guide"] = testing_examples_df.loc[testing_examples_df.gen_test_guide.str.len().eq(1), "gen_test_guide"].str[0]
# testing_examples_df["result_guide"] = testing_examples_df.gen_test_guide.str.split("Step 5: Represent the logical form").str[-1].str.replace("</s>", "").str.strip()
# testing_examples_df["guide_correct"] = testing_examples_df.apply(lambda row: recogs_exact_match(row["output_renum"], row["result_guide"]), axis=1)
# testing_examples_df.to_parquet(f"{storage_path}/model_guide_170_63_preds.parquet")
# testing_examples_df["guide_correct"].mean()